In [ ]:
from sklearn.metrics import f1_score, classification_report
import pathlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from itertools import chain, combinations

import seaborn as sns
import collections
import numpy as np
import pandas as pd
import importlib
import math
import time
import json
import pyperclip
import utils
import categorization
import scoring
import adapted_metrics

importlib.reload(categorization)
importlib.reload(scoring)
importlib.reload(adapted_metrics)

from utils import *
from categorization import *
from scoring import get_N_score
from adapted_metrics import * 

current_path = pathlib.Path().resolve().parent
print(current_path)

json_files_path = current_path / 'similarity_among_tokens_json_files'

In [ ]:
df = pd.read_csv(current_path / 'nbs' / 'partial_match_data2.csv')

df

In [ ]:
df_with_similarities = df.copy()

tau = 0.95
tauprime = 0.95

start_time = time.time()

for i, row in df_with_similarities.iterrows():
    train_run = row['run']
    dataset = row['dataset']
    sts_type = row['sts_type']

    arg_orig = str(row['gold_argument']).split()
    arg_pred = str(row['predicted_argument']).split()
    arg_pred_ids = str(row['pID'])

    number_tokens_pred = get_number_of_tokens(arg_pred)
    number_tokens_gold = get_number_of_tokens(arg_orig)

    with open(json_files_path / f'comparison{i+1}-{dataset}-{sts_type}-{train_run}.json', 'r') as f:
        comparison_dict = json.load(f)


    similarity_tokens = get_similarity_tokens(arg_pred, arg_orig, tauprime, tau, comparison_dict)

    number_similarity_tokens = len(similarity_tokens)

    df_with_similarities.at[i, 'adapted_jaccard'] = adapted_jaccard(number_tokens_pred, number_tokens_gold, number_similarity_tokens)

    df_with_similarities.at[i, 'adapted_dice'] = adapted_dice(number_tokens_pred, number_tokens_gold, number_similarity_tokens)

    df_with_similarities.at[i, 'adapted_sorensen'] = adapted_sorensen(number_tokens_pred, number_tokens_gold, number_similarity_tokens)

    df_with_similarities.at[i, 'adapted_symmetric_anderberg'] = adapted_symmetric_anderberg(number_tokens_pred, number_tokens_gold, number_similarity_tokens)

    df_with_similarities.at[i, 'adapted_sokal_sneath2'] = adapted_sokal_sneath2(number_tokens_pred, number_tokens_gold, number_similarity_tokens)

    df_with_similarities.at[i, 'adapted_ochiai'] = adapted_ochiai(number_tokens_pred, number_tokens_gold, number_similarity_tokens)

    df_with_similarities.at[i, 'adapted_kulczynski2'] = adapted_kulczynski2(number_tokens_pred, number_tokens_gold, number_similarity_tokens)

    elapsed_time = time.time() - start_time
    if (i+1) % 10000 == 0:
        print(f"{i+1} completed [{round(elapsed_time)}]")

df_with_similarities.to_csv(current_path / 'nbs' / f'partial_match_data_with_similarities-{tau}-{tauprime}.csv', index = False)

In [ ]:
df_with_similarities

In [ ]:
# df_with_similarities.to_csv(current_path / 'nbs' / 'partial_match_data_with_similarities-09-09-check.csv', index = False)

# Results
<!--  -->

In [ ]:
tau = 0.8
tauprime = 0.8

df = pd.read_csv(current_path / 'nbs' / f'partial_match_data_with_similarities-{tau}-{tauprime}.csv')

print(df.columns)

df

## N Score (using similarity measures)

In [ ]:
def get_partial_match_values_for_similarity_metric(subdf, target_measure_name, nrun):
    # target_measure_name = f'adapted_{similarity_measure}'
    subdf = subdf[subdf['run'] == nrun]
    means = subdf.groupby("pID")[target_measure_name].mean().tolist()

    return means
    

In [ ]:
dataname = 'ugen'
dataname = 'pe'
dataname = 'webis'

target_column = 'adapted_jaccard'
target_column = 'adapted_dice'
target_column = 'adapted_sorensen'
target_column = 'adapted_symmetric_anderberg'
target_column = 'adapted_sokal_sneath2'
target_column = 'adapted_ochiai'
target_column = 'adapted_kulczynski2'

tanda, sts_type = 'simple', 'no_sts'
tanda, sts_type = 'tanda-2', 'sts_sbert'
tanda, sts_type = 'tanda-2', 'sts_sbert_ft'
tanda, sts_type = 'tanda-2', 'sts_arguebert_ft'

best_f1 = 0
best_model = 0

lambda_PM = 1
lambda_MU = 1
lambda_UNR = 1

N_scores = []


for i in range(10):
    dirpath = str(current_path / f'results-{tanda}' / f'{dataname}_{sts_type}' / f'test_{dataname}_{i}.txt')
    tokens, labels, predictions = parse_file(dirpath)
    run_f1 = f1_score(labels, predictions, average='macro')
    if (run_f1 > best_f1):
        best_model = i
        best_f1 = run_f1
    
    instances = parse_file_for_arg_level(dirpath)
    id_values_pairs = get_values_for_predicted_arguments(instances)

    number_match, number_partial_match, number_made_ups = get_number_predicted_arguments_per_category(id_values_pairs)
    number_unrecognized = get_unrecognized_arguments(instances)
        
    number_golds = get_gold_arguments(instances)
    number_preds = get_pred_arguments(instances)
        
    assert (number_match + number_partial_match + number_made_ups) == number_preds

    subdf = df[(df['dataset'] == dataname) & (df['sts_type'] == sts_type)]
    
    similarity_values_for_no_match = get_partial_match_values_for_similarity_metric(subdf, target_column, i)


    N_score = get_N_score(number_match, number_partial_match, number_made_ups, number_unrecognized, similarity_values_for_no_match, 
                          number_golds, number_preds, 
                          lambda_PM, lambda_MU, lambda_UNR)

    print(N_score)

    N_scores.append(N_score)

print(best_model)


print("BEST score: ", round(N_scores[best_model], 3))

print("AVG score: ", round(np.mean(N_scores), 3))

slist_as_string = f"{round(N_scores[best_model], 3)} ({round(np.mean(N_scores), 3)}) & "
pyperclip.copy(slist_as_string)

print(slist_as_string)
